</div>
<div align="left">
  <img src="img/abstract.png" width="400" alt="Funny little diagram">
  <p><em> Evolve nodes, evolve plans, and learn from the best performing ones.</em></p>
</div>
<div align="center">
</em></p>
</div>

#### Node Initialization (Refactoring ...)

In [1]:
from methods.llm import get_vllm_endpoint_func
from methods.llm import get_claude_response


get_claude_response("hi")



# Own served instance (vLLM, much faster)
# model_name = "unsloth/Meta-Llama-3.1-8B-Instruct"
# INTERNAL_PORT = 30000
# POD_ID = "ppas13gg82lsdh"
# get_endpoint_response = get_vllm_endpoint_func(model_name, POD_ID, INTERNAL_PORT)


# from methods.llm import get_async_vllm_endpoint
# import os 

# # Old serverless Runpod endpoint (slower and steal from us ...)
# endpoint_id = "vllm-rimbwy29f75muq"
# api_key = os.environ["RUNPOD_API_KEY"]
# get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

🥤 vLLM not available - running in fallback mode without GPU/CUDA


'Hello! How can I help you today?'

In [2]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.evolnode import EvolNode
from methods.llm import get_groq_response, get_claude_response

# Code + Compilor Task
# mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)
# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

test_inputs = [c[0] for c in test_cases]

node = EvolNode(mp, None, None, get_response=get_claude_response, test_cases=test_cases) # setting manual test cases

node.evolve("i1", replace=True, batch_size=4, num_runs=1, print_summary=True) # Scale up batch size


# input_dict = {"name": "Dilireba"}
# node.switch_engine(get_groq_response) # switch to grog inference engine 
# output_dict = node(input_dict) # use node as a function || To test on structured output here (constrained decoding)
# print("Output dict: ", output_dict)

/Users/fangyuanyu/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


     :: Query time: 3.41s
string indices must be integers, not 'str'
     :: Evolution time: 34.27s


Concurrent Execution of 8 Node functions to generate prompts ...: 100%|██████████| 8/8 [00:01<00:00,  5.42it/s]


string indices must be integers, not 'str'

Code 1 outputs:
Test 0: {'age': 31}
Test 1: {'age': 26}

Code 0 outputs:
Test 1: {'age': 26}
Test 0: {'age': 31}

Code 2 outputs:
Test 1: {'age': 26}
Test 0: {'age': 32}

Code 3 outputs:
Test 1: {'age': 26}
Test 0: {'age': 31}
list index out of range
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.33
  🎯 Functional fitness: 1.00
  ⭐ Global fitness:     0.67
  🔄 Compiled solutions:        4
  ⏱️ Time breakdown:
     :: Query time: 3.41s
     :: Evolution time: 34.27s
     :: Evaluation time: 79.51s
     :: Total time: 117.20s


📊 Code 0: Fitness: 41.7%
--------------------------------------------------------------------------------
❌ Error Messages:
Input: {'name': 'Dilireba'}, prediction is not aligned with expected output, Expected: {'age': 32} Predicted: {'age': 31}, Error message: 
Value mismatch for key age: 31 != 32




📊 Code 1: Fitness: 41.7%
--------------------------------------------------------------------------------
❌

[]

In [17]:
def generate_prompt(name: str) -> str:
    # search_result = search_google(f'{name} age birth date')
    search_result = "32"
    prompt = f"""Using the following search result about {name}, extract their current age. Only provide:
1. A brief reasoning for the age calculation (1 sentence)
2. A JSON response in markdown format with the age as an integer

Search result: {search_result}

REQUIRED OUTPUT STRUCTURE (or people will die):
- One sentence reasoning
- JSON response in markdown format like this {{"age": <integer>}}

DO NOT include any other information or formats in your response."""
    return prompt

def generate_grammar(meta_prompt) -> str:
    meta_prompt.outputs
    meta_prompt.output_types
    grammar_str = f"""
json 
"""


prompt = generate_prompt("Dilireba")

In [18]:
print(prompt)

grammar = f"""

"""

Using the following search result about Dilireba, extract their current age. Only provide:
1. A brief reasoning for the age calculation (1 sentence)
2. A JSON response in markdown format with the age as an integer

Search result: 32

REQUIRED OUTPUT STRUCTURE (or people will die):
- One sentence reasoning
- JSON response in markdown format like this {"age": <integer>}

DO NOT include any other information or formats in your response.


In [20]:
response = get_claude_response(prompt)

grammar_str = get_grammar(meta_prompt)

get_cfg_response(prompt, grammar_str)

print(response)

'[a-z]'

Since Dilireba's age is directly stated as 32 in the search result, this is her current age.

```json
{"age": 32}
```


In [21]:
from methods.evolnode import extract_json_from_text

extract_json_from_text(response)

{'age': 32}

In [10]:
print(node.reasoning)

To get a celebrity's age accurately, we need to search for current information using Google and have the AI process the search result into a structured format


</div>
<div align="center">
  <img src="img/Project-Nirvana-evolve.gif" width="500" alt="Fourier reconstruction convergence">
  <p><em> Evolve a population of nodes. </em></p>
</div>

In [ ]:
# Population building phase ... 
from methods.llm import get_groq_response, get_claude_response
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.population import Evolution

mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

evo = Evolution(pop_size=20, meta_prompt=mp, get_response=get_endpoint_response, 
                test_cases=test_cases, max_attempts=3, num_eval_runs=2,
                load=True)

strategies = ["m2"] # ["i1", "i1", "m2", "e2"]
evo.get_offspring(strategies)

evo.chat("How effective is the current evolution strategy? What improvement has it made in terms of fitness, and in terms of the implementation?",
         get_claude_response) 

# code-based check 
print(evo.population_info)

Processing queries: 100%|██████████| 20/20 [00:16<00:00,  1.20it/s]


 :: Total time elapsed: 16.71s, 0 errors


Processing queries: 100%|██████████| 34/34 [00:28<00:00,  1.19it/s]


 :: Total time elapsed: 28.46s, 0 errors


Processing queries: 0it [00:00, ?it/s]


 :: Total time elapsed: 0.00s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.33
  🎯 Functional fitness: 1.00
  ⭐ Global fitness:     0.67
  🔄 Batch size:        17
  ⏱️ Time taken: 85.64 seconds


📊 Code 0: Fitness: 16.7%
--------------------------------------------------------------------------------
❌ Error Messages:
Input: {'name': 'Dilireba'}, prediction is not aligned with expected output, Expected: {'age': 32} Predicted: {'age': 87}, Error message: 
Value mismatch for key age: 87 != 32


Input: {'name': 'ChengXiao'}, prediction is not aligned with expected output, Expected: {'age': 26} Predicted: {'age': 33}, Error message: 
Value mismatch for key age: 33 != 26




📊 Code 1: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Failed to parse LLM response -- No JSON structure found in the provided text.
Failed to parse LLM response -- JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstL


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="900" alt="Fourier reconstruction convergence">
  <p><em> Plan, and evolve the plans. </em></p>
</div>


In [ ]:
from methods.llm import get_claude_response, get_groq_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan
from methods.evolnode import PlanNode


# Initialize PlanNode 
mp = MetaPlan("Collect AI papers from arXiv in the given time frame and save it in output directory", "get_arxiv_papers", ["start_date", "end_date", "output_dir"], ["num_papers", "time_taken"], ["str", "str", "str"], ["int", "int"])
plan = EvolNode(mp, get_endpoint_response)

# i1 evolution of plan
plan_dicts, err_msg = plan.evolve_plan_dict(method="i1", batch_size=100) # Batch_size of 100 gives no slow-down

# for plan_dict in plan_dicts:
#     visualize_plan_dict(plan_dict, plan.meta_prompt.task)
#     break

# Manual input on main-node test cases 
main_test_cases = [
    ({"start_date": "2024-11-17", "end_date": "2024-11-23", "output_dir": "tmp"}, {"num_papers": 194, "time_taken": 480})
]


plan.spawn_test_cases_majority(main_test_cases)
plan.evolve_sub_nodes(batch_size=20)

In [ ]:
# plan.evolve_sub_nodes() # Completely stuck in the first call, debugging ... 

from methods.evolnode import EvolNode
from methods.meta_prompt import MetaPrompt, PromptMode

self = plan 
for i, node_dict in enumerate(self.plan_dict["nodes"]):
    meta_prompt = MetaPrompt(
        task=node_dict["task"],
        func_name=node_dict["name"],
        inputs=node_dict["inputs"],
        outputs=node_dict["outputs"],
        input_types=node_dict["input_types"],
        output_types=node_dict["output_types"],
        mode=PromptMode((node_dict.get("mode", "code")).lower())
    )
    test_cases = self.test_cases_dict[node_dict["name"]]
    if "fitness" in node_dict and "code" in node_dict: 
        node = EvolNode(meta_prompt, node_dict["code"], node_dict["reasoning"], get_response=self.get_response, test_cases=test_cases, fitness=node_dict["fitness"])
    else:
        node = EvolNode(meta_prompt, None, None, get_response=self.get_response, test_cases=test_cases)
        print(f"🎲 :: Evolving {node.meta_prompt.func_name} ... ({i+1}/{len(self.plan_dict['nodes'])})")
        node.evolve("i1", replace=True, max_tries=2, num_runs=2, batch_size=20) # It's funny how 30+ sec could elapse before llm inference ... (collecting prompts ?? wtf is taking so long ??)
    self.nodes.append(node)

🎲 :: Evolving extract_age ... (2/2)


Processing LLM queries: 100%|██████████| 20/20 [00:21<00:00,  1.06s/it]


 :: Total time elapsed: 21.23s, 0 errors


Processing LLM queries: 0it [00:00, ?it/s]

 :: Total time elapsed: 0.00s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.00
  🎯 Functional fitness: 0.00
  ⭐ Global fitness:     0.00
  🔄 Compiled solutions:        20
  ⏱️ Time breakdown:
     :: Query time: 2.72s
     :: Evolution time: 21.30s
     :: Evaluation time: 0.03s
     :: Total time: 24.06s


📊 Code 0: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Missing required input parameters: text
Missing required input parameters: text


📊 Code 1: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Missing required input parameters: text
Missing required input parameters: text


📊 Code 2: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Missing required input parameters: text
Missing required input parameters: text


📊 Code 3: Fitness: 0.0%
----------------------------

In [10]:
visualize_plan_dict(plan.plan_dict, plan.meta_prompt.task)

success: successfully compiled d2_output/plan_graph.d2 to d2_output/plan_graph.png in 170.887ms
